# Exploring Chinook Database

## Schema Diagram for the Chinook database 

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "../../Images/chinook-schema.svg")

In [2]:
import mysql.connector

In [3]:
config = {
    'user':'root',
    'password':'password',
    'host':'localhost',
    'database':'',
    'port':'3306',
}

In [4]:
def connect(config):
    return mysql.connector.connect(**config)

In [5]:
cnx = connect(config)
cursor = cnx.cursor()

In [6]:
q = "USE Chinook"
cursor.execute(q)

In [7]:
q = "SHOW tables"
cursor.execute(q)
cursor.fetchall()

[('Album',),
 ('Artist',),
 ('Customer',),
 ('Employee',),
 ('Genre',),
 ('Invoice',),
 ('InvoiceLine',),
 ('MediaType',),
 ('Playlist',),
 ('PlaylistTrack',),
 ('Track',)]

In [8]:
q = "DESCRIBE Track"
cursor.execute(q)
cursor.fetchall()

[('TrackId', 'int', 'NO', 'PRI', None, ''),
 ('Name', 'varchar(200)', 'NO', '', None, ''),
 ('AlbumId', 'int', 'YES', 'MUL', None, ''),
 ('MediaTypeId', 'int', 'NO', 'MUL', None, ''),
 ('GenreId', 'int', 'YES', 'MUL', None, ''),
 ('Composer', 'varchar(220)', 'YES', '', None, ''),
 ('Milliseconds', 'int', 'NO', '', None, ''),
 ('Bytes', 'int', 'YES', '', None, ''),
 ('UnitPrice', 'decimal(10,2)', 'NO', '', None, '')]

#### Write a query that gathers data about the invoice with an invoice_id of 4. Include the following columns in order:
* The id of the track, track_id.
* The name of the track, track_name.
* The name of media type of the track, track_type.
* The price that the customer paid for the track, unit_price.
* The quantity of the track that was purchased, quantity.

In [9]:
q = """
SELECT 
t.TrackId track_id,
t.Name track_name,
mt.Name track_type,
il.UnitPrice unit_price,
il.Quantity
FROM InvoiceLine il
INNER JOIN Track t ON t.TrackId = il.TrackId
INNER JOIN MediaType mt ON mt.MediaTypeId = t.MediaTypeId
WHERE il.InvoiceLineId = 4;
"""
cursor.execute(q)
r = cursor.fetchall()

for i in r:
    print(i)

(8, 'Inject The Venom', 'MPEG audio file', Decimal('0.99'), 1)


Find the artist for each track by joining albums and artist table

In [10]:
q = """ 
SELECT
    t.TrackId track_id,
    t.Name track_name,
    ar.Name artist_name,
    mt.Name track_type,
    il.UnitPrice unit_price,
    il.Quantity
FROM InvoiceLine il
INNER JOIN Track t ON t.TrackId = il.TrackId
INNER JOIN MediaType mt ON mt.MediaTypeId = t.MediaTypeId
INNER JOIN Album al ON al.AlbumId = t.AlbumId
INNER JOIN Artist ar ON ar.ArtistId = al.ArtistId
WHERE il.InvoiceId = 4;
"""
cursor.execute(q)
r = cursor.fetchall()

for i in r:
    print(i)

(42, 'Right Through You', 'Alanis Morissette', 'MPEG audio file', Decimal('0.99'), 1)
(48, 'Not The Doctor', 'Alanis Morissette', 'MPEG audio file', Decimal('0.99'), 1)
(54, 'Bleed The Freak', 'Alice In Chains', 'MPEG audio file', Decimal('0.99'), 1)
(60, 'Confusion', 'Alice In Chains', 'MPEG audio file', Decimal('0.99'), 1)
(66, 'Por Causa De Você', 'Antônio Carlos Jobim', 'MPEG audio file', Decimal('0.99'), 1)
(72, 'Angela', 'Antônio Carlos Jobim', 'MPEG audio file', Decimal('0.99'), 1)
(78, 'Master Of Puppets', 'Apocalyptica', 'MPEG audio file', Decimal('0.99'), 1)
(84, 'Welcome Home (Sanitarium)', 'Apocalyptica', 'MPEG audio file', Decimal('0.99'), 1)
(90, 'Set It Off', 'Audioslave', 'MPEG audio file', Decimal('0.99'), 1)


### Find out which artists are purchased the most. 
* Specifically, what we want to produce is a query that lists the top 10 artists, calculated by the number of times a track by that artist has been purchased.

* `In order to run that query I had to SET sql_mode = ""`


In [11]:
q = """
SELECT
    ta.Title album, 
    ta.Name artist,
    COUNT(*) TracksPurchased
FROM InvoiceLine il
INNER JOIN (
            SELECT
                t.TrackId,
                al.Title,
                ar.Name
            FROM Track t
            INNER JOIN Album al ON al.AlbumId = t.AlbumId
            INNER JOIN artist ar ON ar.ArtistId = al.ArtistId
           ) ta
           ON ta.TrackId = il.TrackId
GROUP BY 1
ORDER BY 3 DESC LIMIT 5
"""
cursor.execute(q)
r = cursor.fetchall()

for i in r:
    print(i)

('Minha Historia', 'Chico Buarque', 35)
('Greatest Hits', 'Lenny Kravitz', 26)
('Unplugged', 'Eric Clapton', 25)
('Acústico', 'Titãs', 22)
('Greatest Kiss', 'Kiss', 20)
